# Kết nối Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Đọc dữ liệu

In [ ]:
import pandas as pd

file_name = "/content/drive/MyDrive/Data Mining Field-oriented/data/consolidated/topic/subtopic/Thời_trang_phong_cách.csv"
df = pd.read_csv(file_name, encoding="utf-8-sig")

In [ ]:
print(df.head(3))

     Tài liệu                                           Nội dung  \
0  Tài liệu 1  ngưỡng U40, thành_viên SNSD duy_trì nhan_sắc t...   
1  Tài liệu 3  đây, sưu_tầm móc khoá Labubu trở_thành trào_lư...   
2  Tài liệu 4  1. Pháo 810 điểm Pháo ghi_điểm kết_hợp trang_p...   

                                                Tags            Chủ đề chính  
0  U40,Tiffany,trang phục,tham khảo,dệt kim,trẻ t...  Thời trang, phong cách  
1  Labubu,Huyền My,Lisa,BlackPink,Á hậu,đồ chơi,S...  Thời trang, phong cách  
2  Lọ Lem,Lâm Minh,Ciin,xuyên thấu,húi cua,tạo hì...  Thời trang, phong cách  


# Trích xuất từ khóa cho toàn cụm

## Hàm xóa stop words, số, loại bỏ từ đơn

In [ ]:
import re

def preprocess_text(text):
    text = text.replace(',', ' , ').replace('.', ' . ')
    text = re.sub(r'\b\d+\b', '', text)
    words = text.split()

    processed_words = [' , ' if '_' not in word and word not in {',', '.'} else word for word in words]

    return ' '.join(processed_words)

In [ ]:
preprocess_text("4 phát_động, thi Đại_sứ văn_hoá đọc 2024 thu_hút 1,6 triệu dự thi học_sinh, sinh_viên 9,2 ngàn cơ_sở giáo_dục nước tham_gia. 517 giải cao vòng sơ_khảo 60 tỉnh thành gửi Ban tổ_chức. Kết_quả, Ban tổ_chức trao 4 danh_hiệu Đại_sứ văn_hoá đọc tiêu_biểu 136 giải nhất, nhì, ba, khuyến_khích giải chuyên_đề thí_sinh tham_gia. đó, Đồng_Nai đoạt 2 giải khuyến_khích Cù_Đặng_Bảo_Trân, lớp 84, Trường THTHCSTHPT Song_ngữ Á_Châu Nguyễn_Thái_Bình, lớp 12C1, Trường trung_học_phổ_thông Lê_Quý_Đôn Tân_Mai, thành_phố Biên_Hoà", vietnamese_stopwords)

'phát_động ,  ,  Đại_sứ văn_hoá  ,  thu_hút ,  ,   ,   ,  học_sinh , sinh_viên ,  ,  cơ_sở giáo_dục  ,  tham_gia .  ,   ,   ,  sơ_khảo  ,   ,   ,   ,  tổ_chức . Kết_quả ,  ,  tổ_chức  ,  danh_hiệu Đại_sứ văn_hoá  ,  tiêu_biểu  ,   ,  ,  ,  ,  ,  , khuyến_khích  ,  chuyên_đề thí_sinh tham_gia .  ,  , Đồng_Nai  ,   ,  khuyến_khích Cù_Đặng_Bảo_Trân ,  ,  ,  ,   ,  Song_ngữ Á_Châu Nguyễn_Thái_Bình ,  ,   ,  ,  ,  trung_học_phổ_thông Lê_Quý_Đôn Tân_Mai , thành_phố Biên_Hoà'

## In ra từ khóa quan trọng
t cũng chưa chắc lắm, chắc với mỗi cụm thì sẽ hiệu chỉnh khác nhau

In [ ]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer

text_column = "Nội dung"

texts_in_cluster = df[text_column].dropna().tolist()
texts_in_cluster = [
    preprocess_text(text) for text in texts_in_cluster
]

vectorizer = TfidfVectorizer(max_features=25)
vectorizer.fit(texts_in_cluster)
cluster_vocab = set(vectorizer.get_feature_names_out())

print(f"Vocabulary chung từ TF-IDF (tối đa 100 từ): {cluster_vocab}")

Vocabulary chung từ TF-IDF (tối đa 100 từ): {'hiện_đại', 'việt_nam', 'lựa_chọn', 'màn_hình', 'điện_thoại', 'tính_năng', 'không_gian', 'khách_hàng', 'trang_phục', 'nổi_bật', 'trải_nghiệm', 'cảm_giác', 'ứng_dụng', 'thương_hiệu', 'thiết_kế', 'khả_năng', 'sản_phẩm', 'phong_cách', 'thời_trang', 'công_nghệ', 'dễ_dàng', 'kết_hợp', 'thiết_bị', 'phiên_bản', 'hệ_thống'}


## Xem xét những từ khóa này có thuộc Tags chuẩn không

In [ ]:
import pandas as pd

clean_vocab = {word.replace("_", " ") for word in cluster_vocab}
tags = df["Tags"].dropna().str.split(",").explode().str.strip()

unique_tags = set(tags)

matching_words = clean_vocab & unique_tags

overlap_percentage = len(matching_words) / len(clean_vocab) * 100

print(f"Số từ trùng lặp: {len(matching_words)}")
print(f"Tỷ lệ % trùng lặp: {overlap_percentage:.2f}%")

Số từ trùng lặp: 22
Tỷ lệ % trùng lặp: 88.00%


In [ ]:
import pandas as pd

clean_vocab = {word.replace("_", " ") for word in cluster_vocab}

tags = df["Tags"].dropna().str.split(",").explode().str.strip()

word_counts = {word: tags.str.contains(fr'\b{word}\b', regex=True).sum() for word in clean_vocab}

total_count = sum(word_counts.values())

total_tags = len(tags)
ratio = total_count / total_tags * 100

print("Số lần xuất hiện của từng từ trong clean_vocab:")
for word, count in word_counts.items():
    print(f"'{word}': {count}")

print(f"Tổng số lần các từ trong clean_vocab xuất hiện: {total_count}")
print(f"Tỷ lệ so với tổng số tags: {ratio:.2f}%")

Số lần xuất hiện của từng từ trong clean_vocab:
'thương hiệu': 68
'việt nam': 0
'công nghệ': 40
'khách hàng': 13
'hệ thống': 59
'cảm giác': 5
'thiết kế': 157
'thời trang': 278
'dễ dàng': 13
'không gian': 368
'thiết bị': 63
'điện thoại': 163
'ứng dụng': 48
'kết hợp': 10
'trải nghiệm': 92
'phiên bản': 89
'màn hình': 130
'phong cách': 142
'hiện đại': 48
'lựa chọn': 22
'sản phẩm': 33
'nổi bật': 41
'khả năng': 18
'tính năng': 179
'trang phục': 102
Tổng số lần các từ trong clean_vocab xuất hiện: 2181
Tỷ lệ so với tổng số tags: 3.13%


# Chạy mô hình trích xuất từ khóa

---



## Tính số lượng tags trung bình

In [ ]:
df['Tags'] = df['Tags'].astype(str)

df['Word_Count'] = df['Tags'].apply(lambda x: len(x.split(',')))
average_word_count = df['Word_Count'].mean()

In [ ]:
print(average_word_count)

16.9873417721519


# TextRank

## TextRank

In [ ]:
!pip3 install summa
!pip3 install keybert
!pip3 install gensim
!pip3 install pytextrank

In [ ]:
from summa import keywords
from tqdm import tqdm

results = []
texts_in_cluster = df[text_column].dropna().tolist()

for text in tqdm(texts_in_cluster, desc="Processing Texts"):
    text = preprocess_text(text, vietnamese_stopwords)
    try:
        doc_keywords = keywords.keywords(text, words=16, split=True)
    except Exception as e:
        print(f"Error processing text: {e}")
        doc_keywords = []

    if isinstance(doc_keywords, dict):
        doc_keywords = list(doc_keywords.keys())
    elif not isinstance(doc_keywords, list):
        doc_keywords = []

    for word in cluster_vocab:
        if word in text and word not in doc_keywords:
            doc_keywords.append(word)

    results.append({
        'Text': text,
        'Keywords': ', '.join(doc_keywords)
    })

result_df = pd.DataFrame(results)

Processing Texts:   1%|          | 37/4108 [00:01<02:46, 24.47it/s]

Error processing text: list index out of range


Processing Texts:   1%|          | 47/4108 [00:01<02:21, 28.77it/s]

Error processing text: list index out of range


Processing Texts:   2%|▏         | 70/4108 [00:03<03:20, 20.14it/s]

...

Error processing text: list index out of range


Processing Texts:  99%|█████████▊| 4048/4108 [04:20<00:03, 19.77it/s]

Error processing text: list index out of range
Error processing text: list index out of range


Processing Texts: 100%|██████████| 4108/4108 [04:23<00:00, 15.57it/s]

Error processing text: list index out of range


In [ ]:
def remove_duplicates(keywords):
    unique_keywords = list(dict.fromkeys([word.strip() for word in keywords.split(',')]))
    return ', '.join(unique_keywords)

result_df['Keywords'] = result_df['Keywords'].apply(remove_duplicates)

In [ ]:
print(result_df.head(3))

                                                Text  \
0   ,   ,  , thành_viên  ,  duy_trì nhan_sắc trẻ_...   
1   ,  , sưu_tầm  ,   ,   ,  trào_lưu  ,   ,  ,  ...   
2  .  ,   ,   ,  ghi_điểm kết_hợp trang_phục ấn_t...   

                                            Keywords  
0  trẻ_trung, trang_phục, chân_váy, kết_hợp, tinh...  
1  phiên_bản, huyền_my, phụ_kiện, nhân_vật, độc_đ...  
2  kết_hợp trang_phục ấn_tượng mái_tóc, trào_lưu,...  


In [ ]:
output_file = "/content/drive/MyDrive/Data Mining Field-oriented/output/Thời_trang_phong_cách_with_Keywords_TextRank.csv"
result_df.to_csv(output_file, index=False, encoding='utf-8-sig')

# Đánh giá

## Chuẩn hóa các Keywords và Tags

In [ ]:
try:
    if "Keywords" in result_df.columns:
        result_df["Keywords"] = result_df["Keywords"].str.replace("_", " ")
    else:
        print("The column 'Keywords' does not exist in the DataFrame.")
except Exception as e:
    print(f"Error: {e}")

result_df["Keywords"] = result_df["Keywords"].str.lower()
df["Tags"] = df["Tags"].str.lower()

In [ ]:
print(result_df[["Keywords"]].head())
print(df["Tags"].head())

                                            Keywords
0  trẻ trung, trang phục, chân váy, kết hợp, tinh...
1  phiên bản, huyền my, phụ kiện, nhân vật, độc đ...
2  kết hợp trang phục ấn tượng mái tóc, trào lưu,...
3  carpet check, chi tiết, trang phục, trang sức,...
4  chị em, trang phục, mũi nhọn, búp bê, cảm giác...
0    u40,tiffany,trang phục,tham khảo,dệt kim,trẻ t...
1    labubu,huyền my,lisa,blackpink,á hậu,đồ chơi,s...
2    lọ lem,lâm minh,ciin,xuyên thấu,húi cua,tạo hì...
3    quế anh,mai ngô,carpet check,lăng xê,miss gran...
4    mùa thu,mule,jane,sành điệu,sắm,chị em,giày,xu...
Name: Tags, dtype: object


## Số lượng tag trùng khớp

In [ ]:
total_common_words = 0

In [ ]:
result_df["Keywords"] = result_df["Keywords"].str.lower()
df["Tags"] = df["Tags"].str.lower()

import re

result_df["Keywords"] = result_df["Keywords"].apply(
    lambda x: ", ".join([word for word in re.split(r",\s*", x) if not word.isdigit()]) if pd.notnull(x) else x
)

total_common_words = 0

def compare_keywords_and_tags(keywords, tags):
    global total_common_words
    if pd.isnull(keywords) or pd.isnull(tags):
        return ""
    keywords_set = set(word.strip() for word in keywords.split(","))
    tags_set = set(word.strip() for word in tags.split(","))
    common_words = keywords_set.intersection(tags_set)
    num_common_words = len(common_words)
    total_common_words += num_common_words
    # print("Common Words:", common_words)
    return ", ".join(common_words)

result_df["Common Words"] = result_df["Keywords"].combine(
    df["Tags"], compare_keywords_and_tags
)

print("Total Common Words:", total_common_words)

Total Common Words: 12133


## Micro Metrics

In [ ]:
def calculate_metrics(keywords, tags):
    if pd.isnull(keywords) or pd.isnull(tags):
        return 0, 0, 0, 0, 0, 0

    keywords_set = set(word.strip() for word in keywords.split(","))
    tags_set = set(word.strip() for word in tags.split(","))

    common_words = keywords_set.intersection(tags_set)
    num_common = len(common_words)

    precision = num_common / len(keywords_set) if len(keywords_set) > 0 else 0
    recall = num_common / len(tags_set) if len(tags_set) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    tp = num_common
    fp = len(keywords_set - tags_set)
    fn = len(tags_set - keywords_set)

    return precision, recall, f1, tp, fp, fn

In [ ]:
result_df[["Precision", "Recall", "F1-Score", "TP", "FP", "FN"]] = result_df.apply(
    lambda row: pd.Series(
        calculate_metrics(
            row["Keywords"],
            df.loc[row.name, "Tags"]
        )
    ),
    axis=1
)

total_tp = result_df["TP"].sum()
total_fp = result_df["FP"].sum()
total_fn = result_df["FN"].sum()

micro_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
micro_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
micro_f1 = (2 * micro_precision * micro_recall) / (micro_precision + micro_recall) if (micro_precision + micro_recall) > 0 else 0

print(f"Micro Precision: {micro_precision:.2f}")
print(f"Micro Recall: {micro_recall:.2f}")
print(f"Micro F1-Score: {micro_f1:.2f}")

Micro Precision: 0.17
Micro Recall: 0.17
Micro F1-Score: 0.17
